## Part 3

In [89]:
setwd("/home/leoKraushaar/Documents/School/Year 3/Semester 2/STAT 413/Project/protests/")
set.seed(42)

### Libraries

In [90]:
library(MASS)

### Clean Data

In [91]:
data <- read.csv("data/merged_data.csv")[, -1]
data$food <- NULL
data$manufac <- NULL

In [92]:
newMonth <- function(x) {
    if (x %in% c("December", "January", "February")) {
        return("Winter")
    } else if (x %in% c("March", "April", "May")) {
        return("Spring")
    } else if (x %in% c("June", "July", "August")) {
        return("Summer")
    } else {
        return("Fall")
    }
}

data$month <- sapply(data$month, newMonth)
colnames(data)[2] <- "season"

In [93]:
standardize <- function(x, mu, std) {
    return((x-mu)/std)
}

# data$pop <- sapply(data$pop, function(x) standardize(x, mean(data$pop), sd(data$pop)))

In [94]:
colnames(data)[colnames(data) == "GEO"] <- "prov"

data$prov  <- as.factor(data$prov)
data$season <- as.factor(data$season)
data$year <- as.factor(data$year)

In [95]:
data$retail <- sapply(data$retail, function(x) {standardize(x, mean(data$retail), sd(data$retail))})
data$pop <- sapply(data$pop, function(x) {standardize(x, mean(data$pop), sd(data$pop))})
# data$power <- sapply(data$power, function(x) {standardize(x, mean(data$power), sd(data$power))})
# data$oil <- sapply(data$oil, function(x) {standardize(x, mean(data$oil), sd(data$oil))})

In [96]:
# data$retail <- NULL
head(data)

,year,season,prov,pop,protests,retail,oil,power
,<fct>,<fct>,<fct>,<dbl>,<int>,<dbl>,<int>,<int>
1,2022,Spring,Alberta,0.3320973,17,0.4154217,3983,6069621
2,2022,Spring,British Columbia,0.5216659,42,0.5517610,77433,5240902
3,2022,Spring,Manitoba,-0.3710644,2,-0.4144921,6290,2168371
4,2022,Spring,New Brunswick,-0.5090145,5,-0.5188768,1818,1171958
5,2022,Spring,Newfoundland and Labrador,-0.5713185,2,-0.5779367,77160,686123
6,2022,Spring,Northwest Territories,-0.6820639,0,-0.6965524,0,58889


### Build Model

In [103]:
init_model <- glm.nb(protests ~ season + prov + oil*prov, data=data)

summary(step(init_model))
# summary(init_model)

Warning message:
“glm.fit: fitted rates numerically 0 occurred”


Start:  AIC=1593.22
protests ~ season + prov + oil * prov

           Df Deviance    AIC
- prov:oil 11   356.77 1585.6
<none>          342.41 1593.2
- season    3   386.28 1631.1

Step:  AIC=1585.46
protests ~ season + prov + oil

         Df Deviance    AIC
- oil     1   349.98 1584.1
<none>        349.33 1585.5
- season  3   394.23 1624.4
- prov   12  1381.38 2593.5

Step:  AIC=1584.11
protests ~ season + prov

         Df Deviance    AIC
<none>        348.80 1584.1
- season  3   396.76 1626.1
- prov   12  2010.84 3222.1



Call:
glm.nb(formula = protests ~ season + prov, data = data, init.theta = 8.079867033, 
    link = log)

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                    2.81967    0.10470  26.931  < 2e-16 ***
seasonSpring                  -0.03746    0.08205  -0.457   0.6480    
seasonSummer                  -0.54781    0.08704  -6.294 3.10e-10 ***
seasonWinter                  -0.18977    0.08757  -2.167   0.0302 *  
provBritish Columbia           0.62564    0.12503   5.004 5.62e-07 ***
provManitoba                  -0.33500    0.13541  -2.474   0.0134 *  
provNew Brunswick             -0.83074    0.14533  -5.716 1.09e-08 ***
provNewfoundland and Labrador -1.15759    0.15449  -7.493 6.74e-14 ***
provNorthwest Territories     -3.22119    0.30203 -10.665  < 2e-16 ***
provNova Scotia               -0.70980    0.14252  -4.980 6.34e-07 ***
provNunavut                   -2.84012    0.25844 -10.989  < 2e-16 ***
provOntario                

## Perform Monte Carlo Simulation

### Get an idea of retail and population

In [98]:
meanretail <- aggregate(data[c("pop")], by=list(prov=data$prov), FUN=mean)
meanretail

prov,pop
<fct>,<dbl>
Alberta,0.3551965
British Columbia,0.5450133
Manitoba,-0.3657537
New Brunswick,-0.5052908
Newfoundland and Labrador,-0.5702505
Northwest Territories,-0.6820850
Nova Scotia,-0.4554273
Nunavut,-0.6830511
Ontario,2.8100380
